#Files to migrate and anonymize:

Action Items:
* Created Submitted folder (notify the team) (Tes)
* Create global patients mapping and use for trainting and test phase data(Tes)
* Team will move the model artificates(all)
* Create a new folder REwithDate(Tes)
* Syncup the common data preprocessing pipeline and generate the correct dataset(XYZ/LL)
* Creating the anonymized data of mappings(Tes)
* Cleanup local copies of raw data(all)
* Replace all the patient ids with the anonymized ids for all data (all)


* readmissions_2011
* readmissions_2012



In [2]:
import os
import json
import random
import string
import pandas as pd

In [3]:
data_path = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/anonymize/REWithDates/Data/Raw/365TestPhase/readmission_targets_with_date_testdata.csv'

In [6]:
df = pd.read_csv(data_path, low_memory=False)
print(df.shape)
df.head()

(1111571, 10)


,admit_dt,disch_dt,events,unplanned,Days_since_last_discharge,first,unplanned_readmission,discharge_id,patient_id,unplanned_readmission_date
0,2012-05-23,2012-05-26,"['d_04189', 'd_311', 'd_99859']",True,NaN,True,False,499999951_20120526,499999951,NaN
1,2011-01-31,2011-02-03,"['d_V1251', 'd_71535', 'd_78900']",True,NaN,True,False,100000005_20110203,100000005,NaN
2,2011-05-03,2011-05-05,"['d_45341', 'd_72400', 'd_4019']",True,NaN,True,False,100000203_20110505,100000203,NaN
3,2012-01-05,2012-01-14,"['d_42789', 'd_41071', 'd_41401']",True,NaN,True,False,100000221_20120114,100000221,NaN
4,2011-01-17,2011-01-25,"['d_8052', 'd_5990', 'd_42831']",True,NaN,True,False,100000315_20110125,100000315,NaN


In [ ]:
def get_csv_fnames(data_dir):
    """Get all csv filenames from directory."""
    fnames = os.listdir(data_dir)
    fnames = [fname for fname in fnames if fname.endswith('.csv')]
    fnames = sorted(fnames)
    return fnames


def get_fpaths(data_dir, data_types):
    """Get all csv filenames from directory."""
    fpaths = []
    for data_type in data_types:
        my_data_dir = os.path.join(data_dir, data_type)
        fnames = os.listdir(my_data_dir)
        fnames = [fname for fname in fnames if fname.endswith('.csv')]
        fnames = sorted(fnames)
        fpaths += [os.path.join(my_data_dir, fname) for fname in fnames]
    return fpaths


def get_unique_values(fpaths, column):
    """Get all unique values from all files in a given column."""
    print('Getting unique values for the {} column...'.format(column))
    values = set()
    for fpath in fpaths:
        print('Processing {}...'.format(fpath))
        df = pd.read_csv(fpath, low_memory=False)
        ids = set(df[column].tolist())
        values.update(ids)
    print('Success!')
    return list(values)


def _generate_strings_v0(num_values=1000, length=16):
    """Generate random strings for patient ids."""
    def _generate_unique_string(generated_list, length):
        """
        Generate unique random string for patient id
        Source: https://stackoverflow.com/questions/2511222/efficiently-generate-a-16-character-alphanumeric-string
        """
        #x = ''.join(random.choices(string.ascii_letters + string.digits, k=length))
        x = ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))
        while x in generated_list:
            x = ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))
        generated_list.append(x)
        return generated_list

    gen_list = []
    for i in range(num_values):
        gen_list = _generate_unique_string(gen_list, length)
    return gen_list

def _generate_strings(num_values=1000, length=16):
    """Generate random strings for patient ids."""
    def _generate_unique_string(generated_set, length):
        """
        Generate unique random string for patient id
        Source: https://stackoverflow.com/questions/2511222/efficiently-generate-a-16-character-alphanumeric-string
        """
        #x = ''.join(random.choices(string.ascii_letters + string.digits, k=length))
        x = ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))
        set_len = len(x)
        generated_set.add(x)
        while set_len == len(generated_set):
            x = ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))
            generated_set.add(x)
        return generated_set

    gen_set = set()
    for i in range(num_values):
        gen_set = _generate_unique_string(gen_set, length)
    return list(gen_set)


def get_all_mappings(unique_values, column, output_dir, string_length):
    """Get mappings from the given unique values for a column to be anonymized."""
    print('Getting all mappings of values with randomly generated strings...')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    num_values = len(unique_values)
    rand_values = _generate_strings(num_values, string_length)
    all_mappings = dict(zip(unique_values, rand_values))

    output_path = os.path.join(output_dir, column+'_mappings.json')
    
    if os.path.exists(output_path):
        raise ValueError('Error! {} already exists. Please remove it and try again.'.format(output_path))
        
    with open(output_path, 'w') as fp:
        json.dump(all_mappings, fp)
    print('Success! Mappings saved to {}!'.format(output_path))
    return output_path
    
    
def read_mappings(mappings_path):
    """Read the patients mappings"""
    with open(mappings_path, 'r') as fp:
        mappings = json.load(mappings_path)
    return mappings


def apply_mappings(df0, output_path, all_mappings):
    """Add mapping columns."""
    df = df0.copy(deep=True)
    
    def _map_values(row, mappings, column):
        """Get mapping for a dataframe row's specific column"""
        value = row[column]
        return value

    for column, mappings in all_mappings.items():
        new_values = df.apply(_map_values, axis=1, args=(mappings,column))
        df[column] = new_values
    
    df.to_csv(output_path, index=False)
    
    return df


def anonymize_ae_dataset(data_dir, fnames, output_dir, all_mappings):
    """Anonymize AE dataset."""
    import pdb; pdb.set_trace()
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    for fname in fnames:
        print('Anonymizing {}...'.format(fname))
        data_path = os.path.join(data_dir, fname)
        output_path = os.path.join(output_dir, fname)
        
        df = pd.read_csv(data_path, low_memory=False)
        df = df.iloc[:100, :] #TODO: Remove later
        
        df = apply_mappings(df, output_path, all_mappings)
    print('Dataset Sucessfully Anonymized and Saved to {}!'.format(output_dir))
    

In [ ]:
## AE
ROOT_DIR = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/anonymize/AE/Data/'
RAW_DIR = os.path.join(ROOT_DIR, 'Raw')
OUTPUT_DIR = os.path.join(ROOT_DIR, 'Anonymized')

DATA_TYPES = ['365NoDeath', '365TestPhase']
ANONYMIZED_COLUMN = 'patient_id'
MAPPINGS_PATH = os.path.join(OUTPUT_DIR, '{}_mappings.csv'.format(ANONYMIZED_COLUMN))

RANDOM_STRING_LENGTH = 9

In [ ]:
fpaths = get_fpaths(RAW_DIR, DATA_TYPES)
unique_values = get_unique_values(fpaths, ANONYMIZED_COLUMN)
mappings_path = get_all_mappings(unique_values, ANONYMIZED_COLUMN, OUTPUT_DIR, RANDOM_STRING_LENGTH)

In [ ]:
# RE
ROOT_DIR = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/anonymize/RE/Data/'
RAW_DIR = os.path.join(ROOT_DIR, 'Raw')
OUTPUT_DIR = os.path.join(ROOT_DIR, 'Anonymized')

DATA_TYPES = ['365', '365TestPhase']
MAPPINGS_PATH = os.path.join(OUTPUT_DIR, '{}_mappings.csv'.format(ANONYMIZED_COLUMN))

In [ ]:
fpaths = get_fpaths(RAW_DIR, DATA_TYPES)
unique_values = get_unique_values(fpaths, ANONYMIZED_COLUMN)
mappings_path = get_all_mappings(unique_values, ANONYMIZED_COLUMN, OUTPUT_DIR, RANDOM_STRING_LENGTH)

In [ ]:
#Read the mappings
#mappings = read_mappings(MAPPINGS_PATH)

#Anonymize each AE data file
#anonymize_ae_dataset(RAW_DIR_AE_365, fnames, OUTPUT_DIR_AE_365, mappings)